In [ ]:
# IS362 Final Project
# Ahmad Malik, 12/11/2022

Let's start by importing all the necessary things

In [ ]:
import pandas as pd
import numpy as np
import json
import requests
import urllib
import pandas.io.json
from pandas.io.json import json_normalize
from matplotlib import pyplot as plt

Now we need to get the data. I found DOTA 2's APIs, which has data that would yield useful results

In [ ]:
############ ***Change hero_id in url to hero you're interested in*** ############
############ I like hero no.30, "Witch Doctor"

#Read the API
url = 'https://api.opendota.com/api/scenarios/laneRoles?hero_id=30'
heroes = 'https://api.opendota.com/api/heroes'
#hero_stats = 'https://api.opendota.com/api/heroStats' ### NOT USED ###
response = urllib.request.urlopen(url)
hresponse = urllib.request.urlopen(heroes)
#hsresponse = urllib.request.urlopen(hero_stats)
responseText = response.read()
hresponseText = hresponse.read()
#hsresponseText = hsresponse.read()

Nextly we need to change it into dataframes, so that we can actually do something with it. I have hero stats data, but it's not really needed to make the graphs I want to make.

In [ ]:
#Make Dataframes
data = json.loads(responseText)
hdata = json.loads(hresponseText)

Now the data has to become usable, so we need to normalize it. Afterwards, look through the (now-legible) data, choose a hero, find its ID, and apply it to the url above. I like the witch doctor, which has the id of 30 as shown in the output below. Keep in mind, there are 120 individual heroes to choose from, and each has different statistics.

In [ ]:
#Normalize 
df = json_normalize(data)
hdf = json_normalize(hdata)
#hsdf = json_normalize(hsdata)      ### NOT USED ###
hdf[20:35]                          #prints data about heroes
#print(hdf["localized_name"])        #prints only localized_name column
#heroList = hdf['localized_name'].tolist()  #creates list of heroes
#heroList                                   #prints hero names as a list

Now we need to do something cool. Since I like the witch doctor [id 30], the url looks like this: url = 'https://api.opendota.com/api/scenarios/laneRoles?hero_id=30' There are 4 lanes in each match, and you need to battle through the lanes in order to claim victory. As time progresses, your chances of winning vary, depending on what lane you're in. This data can be found in the above url, and is based on your hero's id.

Let's first look at what data is contained in that link:

In [ ]:
df

There are 5 intervals of time, 4 lanes, a number of wins, and a total number of games played. This means, that for each lane, we can get a win rate % (wins/games * 100), at each time interval.

In [ ]:
for lane in df['lane_role'].unique():
#Define X and Y axis of line graph
  x = (df[df.lane_role == lane]['time'])/60
  y = ((df[df.lane_role == lane]['wins']).astype(float)/(df[df.lane_role == lane]['games']).astype(float))*100
  print('--------------')
  print(lane)
  print('--------------')
  print('Duration (minutes)')
  print(x)
  print('Win Rate (percentage))')
  print(y)

As the results aren't pretty to look at, let's stick them all into a graph for easier comparison

In [ ]:
for lane in df['lane_role'].unique():
#Define X and Y axis of line graph
  x = (df[df.lane_role == lane]['time'])/60
  y = ((df[df.lane_role == lane]['wins']).astype(float)/(df[df.lane_role == lane]['games']).astype(float))*100
  plt.plot(x, y, label="Lane number {}".format(lane))
  plt.ylim(0, 100) 
#plt.title("Lane number {}".format(lane))
plt.xlabel('Game Length (minutes)')
plt.ylabel('Win Rate (percentage)')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

In conclusion, we can see that as a witch doctor, depending on what lane you are in, and what length of time has passed, your chances of winning vary. At 30 minutes into the game you have a high chance of winning if you're in lane 4, but that drops as you get closer to the end of the game. If you start out in lane 2, and choose to remain in lane 2 throughout the game, your chances of winning slowly rise. To see other heroes' win rates, simply find their ID in hdf, apply it to the URL, and run the code again. Yayyy! \o/